In [24]:
__author__ = 'racah'
import h5py
import numpy as np
from operator import mul
import os
import glob
import sys
import time
from multiprocessing import Pool
from functools import partial








#1 is hur
#0 is nhur
class LoadHurricane():
    def __init__(self,batch_size=None, flatten=False, num_ims=None, seed=4):
        self.seed = seed
        self.batch_size = batch_size
        self.num_ims = num_ims
        self.seed = seed
        self.flatten=flatten
    #TODO try on 96x96 (use bigger file -> get from cori)
    def load_hurricane(self, path, use_negative=True):
    
        print 'getting data...'
        h5f = h5py.File(path)
        if not self.num_ims:
            hurs = h5f['hurricane'][:]
            nhurs = h5f['nothurricane'][:]
            hur_boxes = h5f['hurricane_box'][:]
        else:
            hurs = h5f['hurricane'][:self.num_ims / 2]
            nhurs = h5f['nothurricane'][:self.num_ims / 2]
            hur_boxes = h5f['hurricane_box'][:self.num_ims / 2]
            
        hurs_bboxes = np.asarray(hur_boxes).reshape(hurs.shape[0],4)
        nhurs_bboxes = np.zeros((nhurs.shape[0],4))

        if use_negative:
            inputs = np.vstack((hurs,nhurs))
            bboxes = np.vstack((hurs_bboxes,nhurs_bboxes))
        else:
            inputs = hurs
            bboxes = hurs_bboxes

 
        cl_labels = np.zeros((inputs.shape[0]))
        cl_labels[:hurs.shape[0]] = 1
        if not self.num_ims:
            self.num_ims = inputs.shape[0]

        print self.num_ims


        tr_i, te_i, val_i = self.get_train_val_test_ix(self.num_ims)

        return self.set_up_train_test_val(inputs, bboxes, cl_labels, tr_i, te_i, val_i)



    def get_train_val_test_ix(self, num_ims):
        # tr, te, val is 0.6,0.2,0.2
        ix = range(num_ims)

        n_te = int(0.2*num_ims)
        n_val = int(0.25*(num_ims - n_te))
        n_tr =  num_ims - n_te - n_val


        #shuffle once deterministically
        np.random.RandomState(3).shuffle(ix)
        te_i = ix[:n_te]
        rest = ix[n_te:]

        np.random.RandomState(self.seed).shuffle(rest)
        val_i = rest[:n_val]
        tr_i = rest[n_val:n_val + n_tr]
        return tr_i, te_i, val_i


    def set_up_train_test_val(self,hurs, boxes,cl_labels, tr_i,te_i, val_i):

        x_tr, bbox_tr, lbl_tr = hurs[tr_i], boxes[tr_i], cl_labels[tr_i]
        x_tr, tr_means, tr_stds = self.preprocess_each_channel(x_tr)
        #self.test_masks(bbox_tr, y_tr,np.random.randint(x_tr.shape[0]))
        x_te,bbox_te, lbl_te = hurs[te_i], boxes[te_i], cl_labels[te_i]
        x_te, _ ,_ = self.preprocess_each_channel(x_te)

        #self.test_masks(bbox_te, y_te,np.random.randint(x_te.shape[0]))
        x_val, bbox_val, lbl_val = hurs[val_i], boxes[val_i], cl_labels[val_i]
        x_val, _ ,_ = self.preprocess_each_channel(x_val)
        #self.test_masks(bbox_val, y_val,np.random.randint(x_val.shape[0]))

        if self.flatten:
            x_tr = x_tr.reshape(x_tr.shape[0], reduce(mul, x_tr.shape[1:]))
            x_te = x_te.reshape(x_te.shape[0], reduce(mul, x_te.shape[1:]))
            x_val = x_val.reshape(x_val.shape[0], reduce(mul, x_val.shape[1:]))

        x_dims = hurs.shape[1:]

        return {'tr': (x_tr, bbox_tr, lbl_tr), \
        'te':(x_te,  bbox_te, lbl_te), \
        'val': (x_val ,bbox_val, lbl_val)}
        # return {'x_train': x_tr, 'y_train': y_tr, 'x_test': x_te, 'y_test': y_te,'x_val':x_val, 'y_val':y_val, 'boxes': boxes}

    def preprocess_each_channel(self,arr, means=[], stds=[], mode='normalize'):
        # assumes channels are on the axis 1
        if len(means) == 0:
            means = np.mean(arr, axis=(0, 2, 3))
        if mode == 'standardize':
            if len(stds) == 0:
                stds = np.std(arr, axis=(0, 2, 3))
            for channel, (mean, std) in enumerate(zip(means, stds)):
                arr[:, channel, :, :] -= mean
                arr[:, channel, :, :] /= std
            return arr, means, stds
        elif mode == "normalize":

            for channel, mean in enumerate(means):
                arr[:, channel, :, :] -= mean
                
            mins = np.min(arr, axis=(0, 2, 3))
            maxes = np.max(arr, axis=(0, 2, 3))
            #normalize between -1 and 1
            for channel, (min_,max_) in enumerate(zip(mins, maxes)):
                arr[:, channel, :, :] = 2 * ((arr[:, channel, :, :] - min_) / (max_ - min_)) - 1
            return arr, None, None

                
            
       
        

    def get_cooords(self, bbox):
        hmin, hmax, wmin, wmax = bbox[1],bbox[3], bbox[0],bbox[2]
        return hmin, hmax, wmin, wmax

    
#TODO: load a classification dataset and a localization one   
def load_dataset(num_ims=None, path='/global/project/projectdirs/nervana/yunjie/dataset/localization/larger_hurricanes_loc.h5'):
    lh = LoadHurricane(num_ims=num_ims)
    dataset_dict = lh.load_hurricane(path, use_negative=True)
    
    datasets = [(dataset_dict[k][0],dataset_dict[k][2]) for k in ['tr', 'val','te']]
    (X_train, y_train), (X_val, y_val), (X_test, y_test) = datasets

    y_train, y_val, y_test = [y.astype('int32') for y in [y_train, y_val, y_test]]
    
    return X_train, y_train, X_val, y_val, X_test, y_test




In [25]:
if __name__ == "__main__":
    X_train, y_train, X_val, y_val, X_test, y_test=load_dataset(num_ims=40)

    import matplotlib

    %matplotlib inline

    from matplotlib import pyplot as plt

    print y_train

    for i in range(20):
        g = plt.subplot(5,4,i+1)
        g.set_xlabel(str(y_train[i]))
        g.imshow(X_train[i][0])


getting data...
[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.]
40
